In [7]:
import csv
import re
from os import walk
import functools

def printTotalVotes(totalVotes, init, end):
    print("\\begin{tabular}{" + "|c" * (end - init + 1) + "|}")
    
    print("\\hline")
    
    print("\t", end="")
    
    for i in range(init, end):
        print(" & \\textbf{SDG %d}" % (i + 1), end="")
    
    print("\\\\ \\hline")
    
    print ("\\textbf{Total Evaluations} \t", end="")
    
    for i in range(init, end):
        print(" & %d \t" % (totalVotes[i]), end="")
    
    print("\\\\ \\hline")
    print("\\end{tabular}")
    print("\\\\")
    
def printQGDDTable(qgdd, qgndd, dontKnows, o):

    print("\\begin{tabular}{" + "|c" * (len(qgdd) + 1) + "|}")
    
    print("\\hline")
    print("\t", end="")
    
    for i in range(len(qgdd)):
        print("& " + "\\textbf{R" + str(o) + "\\_" + str(i+1) + "}\t", end="")
        
    print("\\\\ \\hline")
    
    print("\\textbf{QGDD}\t", end="")
    
    for i in range(len(qgdd)):
        print("& " + str(round(qgdd[i], 2)) + "\t", end="")

    print("\\\\ \\hline")
    
    print("\\textbf{QGNDD}\t", end="")
    
    for i in range(len(qgndd)):
        print("& " + str(round(qgndd[i], 2)) + "\t", end="")

    print("\\\\ \\hline")
    
    print("\\textbf{Not Known}\t", end="")
    
    for i in range(len(dontKnows)):
        print("& " + str(dontKnows[i]) + "\t", end="")

    print("\\\\ \\hline")
        
    print("\\end{tabular}")

    
def printRelation(R):
   for i in range(len(R)):
        for j in range(len(R[0])):
            print(str(round(R[i][j], 2)), end="")
            
            if j < len(R[0]) - 1:
                print("\t", end="")
            else:
                print("")

    
'''
$P^{1} = \begin{bmatrix} 
1 & 0\\
0 & -1\\
\end{bmatrix}$
'''
def printRelationAsLatex(R, index):
    print("$P^{" + str(index) + "} = \\begin{bmatrix}")
    
    for i in range(len(R)):
        for j in range(len(R[0])):
            print(str(round(R[i][j], 2)), end="")
            
            if j < len(R[0]) - 1:
                print(" & ", end="")
            else:
                print(" \\\\")
    
    print("\\end{bmatrix}$")

def addMatrices(A, B):
    result = [[0 for col in range(len(A))] for row in range(len(A[0]))]
    
    for i in range(len(A)):
        for j in range(len(A[0])): 
            result[i][j] = A[i][j] + B[i][j] 
    
    return result

def normalizeRelation(R):
    result = [[0 for col in range(len(R))] for row in range(len(R[0]))]
    
    for i in range(len(R)):
        for j in range(len(R[0])):
            if (i != j):
                result[i][j] = R[i][j] / (R[i][j] + R[j][i])
    
    return result

def getFiles(dir, regex):
    _, _, filenames = next(walk(dir))
    
    votesFiles = []
    
    for f in filenames:
        if re.match(regex, f):
            votesFiles.append(f)

    return votesFiles


def getVotesFiles(dir):
    return getFiles(dir, r"^wikisurvey_.*_votes")

def getNonVotesFiles(dir):
    return getFiles(dir, r"^wikisurvey_.*_nonvotes")

def getOptionCode(optionName):
    return optionName[0:5].strip()

def getAvailableOptions(fileName):
    options = []
    
    firstRow = True
    
    f = open(fileName)
    reader = csv.reader(f, delimiter=',')
    for row in reader: # each row is a list
        if not firstRow:  # Ignore first row (column names)
            winnerOption = getOptionCode(row[4])

            if winnerOption not in options:
                options.append(winnerOption)

            loserOption = getOptionCode(row[6])
            
            if loserOption not in options:
                options.append(loserOption)
            
        firstRow = False
    
    f.close()
    
    options.sort()
    
    return options

def parseVotes(fileName, options):
    nOptions = len(options)
    
    relation = [[0 for col in range(nOptions)] for row in range(nOptions)]

    #print(relation)
    
    firstRow = True
    
    f = open(fileName)
    reader = csv.reader(f, delimiter=',')
    for row in reader: # each row is a list
        if not firstRow:  # Ignore first row (column names)
            if row[15] == "TRUE":
                winner = options.index(getOptionCode(row[4]))
                loser = options.index(getOptionCode(row[6]))
                #print(winner, loser)
                relation[winner][loser] = relation[winner][loser] + 1
            
        firstRow = False
    
    f.close()
    
    return relation


def parseNonVotes(fileName, options):
    nOptions = len(options)
    
    relation = [[0 for col in range(nOptions)] for row in range(nOptions)]

    #print(relation)
    
    firstRow = True
    
    f = open(fileName)
    reader = csv.reader(f, delimiter=',')
    for row in reader: # each row is a list
        if not firstRow:  # Ignore first row (column names)
            if row[0] == "Skip": 
                if row[10] == "like_both" or row[10] == "dislike_both":
                    op1 = options.index(getOptionCode(row[6]))
                    op2 = options.index(getOptionCode(row[8]))
                    
                    relation[op1][op2] = relation[op1][op2] + 0.5
                    relation[op2][op1] = relation[op2][op1] + 0.5
                    
        firstRow = False
    
    f.close()
    
    return relation

def getDontKnows(fileName, options): 
    nOptions = len(options)
    
    dontKnows = [0] * len(options)

    firstRow = True
    
    f = open(fileName)
    reader = csv.reader(f, delimiter=',')
    for row in reader: # each row is a list
        if not firstRow:  # Ignore first row (column names)
            if row[0] == "Skip": 
                op1 = options.index(getOptionCode(row[6]))
                op2 = options.index(getOptionCode(row[8]))
                
                if row[10] == "dont_know_both":
                    dontKnows[op1] = dontKnows[op1] + 1
                    dontKnows[op2] = dontKnows[op2] + 1
                elif row[10] == "dont_know_left":
                    dontKnows[op1] = dontKnows[op1] + 1
                elif row[10] == "dont_know_right":
                    dontKnows[op2] = dontKnows[op2] + 1

        firstRow = False
    
    f.close()
    
    return dontKnows


    
def computeConsistency(P):
    cl_p = 0.0
    
    n= len(P)
    
    for i in range(n):
        for k in range(n):
            if i != k:
                aux1 = 0.0
                aux2 = 0.0
                aux3 = 0.0
                
                for j in range(n):
                    if j != i and j != k:
                        cp_ik_j1 = P[i][j] + P[j][k] - 0.5
                        cp_ik_j2 = P[j][k] - P[j][i] + 0.5
                        cp_ik_j3 = P[i][j] - P[k][j] + 0.5

                        t_ik_j1 = abs(cp_ik_j1 - P[i][k])
                        t_ik_j2 = abs(cp_ik_j2 - P[i][k])
                        t_ik_j3 = abs(cp_ik_j3 - P[i][k])
                        
                        print

                        aux1 = aux1 + t_ik_j1
                        aux2 = aux2 + t_ik_j2
                        aux3 = aux3 + t_ik_j3
                        
                aux1 = aux1 / (n - 2)
                aux2 = aux2 / (n - 2)
                aux3 = aux3 / (n - 2)
                
                ep_ik = (aux1 + aux2 + aux3) / 3
                
                cl_p = cl_p + ep_ik
    
    cl_p = cl_p / (n*n + n)
    
    return 1 - cl_p


def most(i):   # Most quatifier 0.3,0.8 (less than 0.3 -> 0, more than 0.8 -> 1. Between 0.3 and 0.8 linarly between 0 and 1)
    if i < 0.3:
        return 0.0
    
    if i > 0.8:
        return 1.0
    
    i = (i - .3) * 2
    
    return i

def weightingVector(n):
    w = [0] * n
    
    for i in range(1,n+1):
        w[i-1] = most(float(i)/n) - most(float(i-1)/n)
        
    return w


def OWA(values, w):
    values.sort()
    #print(values)
    
    res = 0.0
    
    for i in range(len(values)):
        res = res + values[i] * w[i]
        
    return res
    
def QGDD_i(P, i):
    values = []
    w = weightingVector(len(P) - 1)
    
    for j in range(len(P)):
        if (j != i):
            values.append(P[i][j])
            
    #print(w)
    #print(values)
    
    return OWA(values, w)

def QGDD(P):
    qgdd = []
    for i in range(len(P)):
        qgdd.append(QGDD_i(P,i))
    
    return qgdd

def QGNDD_i(P, i):
    values = []
    w = weightingVector(len(P) - 1)
    
    for j in range(len(P)):
        if (j != i):
            val = P[j][i] - P[i][j]
            
            if(val < 0):
                val = 0
                
            values.append(1-val)
            
    #print(w)
    #print(values)
    
    return OWA(values, w)

def QGNDD(P):
    qgndd = []
    for i in range(len(P)):
        qgndd.append(QGNDD_i(P,i))
    
    return qgndd


def sortingFunction(elem1, elem2):
    if (elem1[1] - elem2[1] > 0.03):
        return -1
    
    if (elem2[1] - elem1[1] > 0.03):
        return 1
    
    if (elem1[2] - elem2[2] > 0.03):
        return -1
    
    if (elem2[2] - elem1[2] > 0.03):
        return 1
    
    return 0
    

    
    
    
    
totalVotes = [0] * 17

    
for o in range(1, 18):
    #print("SDG", o)
    
    dir = "exports/ODS-" + str(o) + "/"

    vFiles = getVotesFiles(dir)
    nvFiles = getNonVotesFiles(dir)

    #print(vFiles)

    options = getAvailableOptions(dir + vFiles[0])
    
    n = len(options)
        
    totalRelation = None

    for f in vFiles:
        fileName = dir + f

        relation = parseVotes(fileName, options)

        #print(relation)

        if totalRelation == None:
            totalRelation = relation
        else:
            totalRelation = addMatrices(totalRelation, relation) 
            
            

    
    # Compute non votes
    
    
    dontKnows = None
    
    for f in nvFiles:
        fileName = dir + f

        relation = parseNonVotes(fileName, options)

        #print(relation)

        totalRelation = addMatrices(totalRelation, relation) 
        
        dn = getDontKnows(fileName, options)
        
        #print(dn)
        
        if dontKnows == None:
            dontKnows = dn
        else:
            for op in range(len(options)):
                dontKnows[op] = dontKnows[op] + dn[op]
                
# Sum up number of votes por this SDG
    
    for i in range(n):
        for k in range(n):
            totalVotes[o-1] = totalVotes[o-1] + totalRelation[i][k]
    
    
    #print(dontKnows)   

    #print("TOTAL", totalRelation)
    #print(totalVotes[o-1])
    
    normalized = normalizeRelation(totalRelation)

    #print("NORMALIZED", normalized)
    #printRelation(normalized)
    
    print("\\begin{center}")
    
    printRelationAsLatex(normalized, o)

    
    CP = computeConsistency(normalized)

    print(" ; $CP^{%i}=%s$" % (o, str(round(CP, 2))))
    
    
    print(" ; $TE^{%i}=%i$" % (o, totalVotes[o-1]))
    
    print("\\\\")
    print("\\vspace{0.2cm}")
    
    qgdd = QGDD(normalized)
    qgndd = QGNDD(normalized)
    
    printQGDDTable(qgdd, qgndd, dontKnows, o)
    print("\\\\")
    print("\\vspace{0.2cm}")
    
    
    

    xxx = [0]*len(options)
    
    for i in range(len(options)):
        xxx[i] = (options[i], qgdd[i], qgndd[i])
        
    #print(xxx)
    xxx = sorted(xxx, key=lambda x: x[1], reverse=True)
    
    ordered = sorted(xxx, key=functools.cmp_to_key(sortingFunction))
    
    print("$\\bm{", end="")
    
    for i in range(len(ordered)):
        if i != 0:
            if (sortingFunction(ordered[i-1], ordered[i]) == 0):
                print(" \sim ", end="")
            else:
                print(" \succ ", end="")
            
        print(ordered[i][0], end="")
        
    print("}$\n\\\\")
    print("\\end{center}")
        
    print("\\vspace{0.4cm}")
    print("\n\n")
        

printTotalVotes(totalVotes, 0, 6)
printTotalVotes(totalVotes, 6, 12)
printTotalVotes(totalVotes, 12, 17)



\begin{center}
$P^{1} = \begin{bmatrix}
0 & 0.47 & 0.67 \\
0.53 & 0 & 0.48 \\
0.33 & 0.52 & 0 \\
\end{bmatrix}$
 ; $CP^{1}=0.89$
 ; $TE^{1}=481$
\\
\vspace{0.2cm}
\begin{tabular}{|c|c|c|c|}
\hline
	& \textbf{R1\_1}	& \textbf{R1\_2}	& \textbf{R1\_3}	\\ \hline
\textbf{QGDD}	& 0.59	& 0.51	& 0.45	\\ \hline
\textbf{QGNDD}	& 0.97	& 0.98	& 0.86	\\ \hline
\textbf{Not Known}	& 11	& 13	& 7	\\ \hline
\end{tabular}
\\
\vspace{0.2cm}
$\bm{R1.1 \succ R1.2 \succ R1.3}$
\\
\end{center}
\vspace{0.4cm}



\begin{center}
$P^{2} = \begin{bmatrix}
0 & 0.65 & 0.5 & 0.25 & 0.71 \\
0.35 & 0 & 0.4 & 0.41 & 0.35 \\
0.5 & 0.6 & 0 & 0.43 & 0.19 \\
0.75 & 0.59 & 0.57 & 0 & 0.55 \\
0.29 & 0.65 & 0.81 & 0.45 & 0 \\
\end{bmatrix}$
 ; $CP^{2}=0.84$
 ; $TE^{2}=290$
\\
\vspace{0.2cm}
\begin{tabular}{|c|c|c|c|c|c|}
\hline
	& \textbf{R2\_1}	& \textbf{R2\_2}	& \textbf{R2\_3}	& \textbf{R2\_4}	& \textbf{R2\_5}	\\ \hline
\textbf{QGDD}	& 0.59	& 0.38	& 0.48	& 0.6	& 0.59	\\ \hline
\textbf{QGNDD}	& 1.0	& 0.76	& 0.94	& 1.0	& 0.96	

0.52 & 0 & 0.67 & 0.43 \\
0.29 & 0.33 & 0 & 0.28 \\
0.59 & 0.57 & 0.72 & 0 \\
\end{bmatrix}$
 ; $CP^{17}=0.98$
 ; $TE^{17}=180$
\\
\vspace{0.2cm}
\begin{tabular}{|c|c|c|c|c|}
\hline
	& \textbf{R17\_1}	& \textbf{R17\_2}	& \textbf{R17\_3}	& \textbf{R17\_4}	\\ \hline
\textbf{QGDD}	& 0.54	& 0.55	& 0.3	& 0.62	\\ \hline
\textbf{QGNDD}	& 0.97	& 0.99	& 0.59	& 1.0	\\ \hline
\textbf{Not Known}	& 1	& 0	& 1	& 0	\\ \hline
\end{tabular}
\\
\vspace{0.2cm}
$\bm{R17.4 \succ R17.2 \sim R17.1 \succ R17.3}$
\\
\end{center}
\vspace{0.4cm}



\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
	 & \textbf{SDG 1} & \textbf{SDG 2} & \textbf{SDG 3} & \textbf{SDG 4} & \textbf{SDG 5} & \textbf{SDG 6}\\ \hline
\textbf{Total Evaluations} 	 & 481 	 & 290 	 & 310 	 & 251 	 & 148 	 & 133 	\\ \hline
\end{tabular}
\\
\begin{tabular}{|c|c|c|c|c|c|c|}
\hline
	 & \textbf{SDG 7} & \textbf{SDG 8} & \textbf{SDG 9} & \textbf{SDG 10} & \textbf{SDG 11} & \textbf{SDG 12}\\ \hline
\textbf{Total Evaluations} 	 & 113 	 & 238 	 & 193 	 & 175 	 